In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
directory = r'C:\Users\Administrator\Desktop\DSP391m_cryptocurrency-price-prediction\data\raw\BTC'


# Convert units K, M, B to number
def convert_to_number(x):
    if isinstance(x, str):
        if x.endswith('K'):
            return float(x[:-1]) * 1e3
        elif x.endswith('M'):
            return float(x[:-1]) * 1e6
        elif x.endswith('B'):
            return float(x[:-1]) * 1e9
        else:
            try:
                return float(x)
            except ValueError:
                return np.nan
    return x

# Process CSV file
def process_csv(file_path):
    try:
        df = pd.read_csv(file_path, header=None)
        df.columns = ['Date', 'Open_label', 'Open', 'High_label', 'High', 'Low_label', 'Low', 
                      'Close_label', 'Close', 'Change_label', 'Change', 'Amplitude_label', 'Amplitude',
                      'MA7_label', 'MA7', 'MA25_label', 'MA25', 'MA99_label', 'MA99',
                      'VolBTC_label', 'VolBTC', 'VolUSDT_label', 'VolUSDT', 'Unknown1', 'Unknown2']
        
        # Select and rename columns
        df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Amplitude', 'MA7', 'MA25', 'MA99', 'VolBTC', 'VolUSDT']]
        df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Amplitude', 'MA(7)', 'MA(25)', 'MA(99)', 'Vol(BTC)', 'Vol(USDT)']
        
        # Process data
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df['Change'] = df['Change'].str.rstrip('%').astype('float') / 100.0
        df['Amplitude'] = df['Amplitude'].str.rstrip('%').astype('float') / 100.0
        df['Vol(BTC)'] = df['Vol(BTC)'].apply(convert_to_number)
        df['Vol(USDT)'] = df['Vol(USDT)'].apply(convert_to_number)
        
        return df.dropna()
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return pd.DataFrame()

# Check for missing dates
def check_missing_dates(df, start_date, end_date):
    date_range = pd.date_range(start=start_date, end=end_date)
    missing_dates = date_range.difference(df['Date'])
    return missing_dates

# Handle missing dates using linear interpolation
def handle_missing_dates(df, missing_dates):
    # Create a DataFrame for missing dates with NaN values
    missing_df = pd.DataFrame(missing_dates, columns=['Date'])
    # Merge with the original DataFrame
    df_with_missing = pd.merge(df, missing_df, on='Date', how='outer', sort=True)
    # Interpolate missing values using linear method
    df_with_missing = df_with_missing.set_index('Date').interpolate(method='linear').reset_index()
    
    return df_with_missing


In [3]:

# Initialize a list to store DataFrames
dfs = []

# Iterate through all CSV files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        df = process_csv(file_path)
        if not df.empty:
            dfs.append(df)

# Combine all DataFrames
combined_df = pd.concat(dfs, ignore_index=True)

# Sort DataFrame by date
combined_df = combined_df.sort_values('Date')

# Filter data by date range
start_date = '2019-12-01'
end_date = '2024-07-01'
filtered_df = combined_df[(combined_df['Date'] >= start_date) & (combined_df['Date'] < end_date)]

# Display result
print("Filtered DataFrame:")
print(filtered_df.head())


Filtered DataFrame:
        Date     Open     High      Low    Close  Change  Amplitude    MA(7)  \
0 2019-12-01  7540.63  7541.85  7210.00  7390.89 -0.0200     0.0440  7409.45   
1 2019-12-02  7391.50  7420.56  7151.10  7294.28 -0.0131     0.0365  7435.84   
2 2019-12-03  7294.42  7400.00  7241.35  7292.71 -0.0002     0.0217  7455.35   
3 2019-12-04  7292.71  7750.00  7067.00  7194.32 -0.0135     0.0937  7410.47   
4 2019-12-05  7194.59  7485.00  7150.00  7389.00  0.0271     0.0466  7406.11   

    MA(25)   MA(99)  Vol(BTC)    Vol(USDT)  
0  8067.15  8881.46   60769.0  446525000.0  
1  7990.27  8852.68   46330.0  337889000.0  
2  7931.03  8821.58   33149.0  242379000.0  
3  7866.43  8791.37   83154.0  608671000.0  
4  7800.41  8767.81   59307.0  434739000.0  


In [4]:
# Check for missing dates
missing_dates = check_missing_dates(filtered_df, start_date, end_date)

print("\nMissing dates:")
if len(missing_dates) > 0:
    print(missing_dates)
else:
    print("No missing dates found.")



Missing dates:
DatetimeIndex(['2020-01-08', '2020-01-20', '2020-02-02', '2020-02-14',
               '2020-02-26', '2020-03-09', '2020-03-21', '2020-04-02',
               '2020-04-14', '2020-04-27', '2020-05-09', '2020-05-21',
               '2020-06-02', '2020-06-14', '2020-06-26', '2020-07-08',
               '2020-07-21', '2020-08-02', '2020-08-14', '2020-08-26',
               '2020-09-07', '2020-09-19', '2020-10-01', '2020-10-14',
               '2020-10-26', '2020-11-07', '2020-11-19', '2020-12-01',
               '2020-12-13', '2020-12-25', '2021-01-07', '2021-01-19',
               '2021-01-31', '2021-02-21', '2021-03-22', '2021-04-19',
               '2021-05-18', '2021-06-15', '2021-07-14', '2021-08-11',
               '2021-09-09', '2021-10-07', '2021-11-05', '2021-12-03',
               '2022-01-01', '2024-02-22', '2024-05-08', '2024-07-01'],
              dtype='datetime64[ns]', freq=None)


In [5]:
filled_df = handle_missing_dates(filtered_df, missing_dates)
print("\nDataFrame after handling missing dates:")
print(filled_df.head())


DataFrame after handling missing dates:
        Date     Open     High      Low    Close  Change  Amplitude    MA(7)  \
0 2019-12-01  7540.63  7541.85  7210.00  7390.89 -0.0200     0.0440  7409.45   
1 2019-12-02  7391.50  7420.56  7151.10  7294.28 -0.0131     0.0365  7435.84   
2 2019-12-03  7294.42  7400.00  7241.35  7292.71 -0.0002     0.0217  7455.35   
3 2019-12-04  7292.71  7750.00  7067.00  7194.32 -0.0135     0.0937  7410.47   
4 2019-12-05  7194.59  7485.00  7150.00  7389.00  0.0271     0.0466  7406.11   

    MA(25)   MA(99)  Vol(BTC)    Vol(USDT)  
0  8067.15  8881.46   60769.0  446525000.0  
1  7990.27  8852.68   46330.0  337889000.0  
2  7931.03  8821.58   33149.0  242379000.0  
3  7866.43  8791.37   83154.0  608671000.0  
4  7800.41  8767.81   59307.0  434739000.0  


In [6]:
missing_dates2 = check_missing_dates(filled_df, start_date, end_date)

print("\nMissing dates:")
if len(missing_dates2) > 0:
    print(missing_dates2)
else:
    print("No missing dates found.")


Missing dates:
No missing dates found.


In [7]:
BTC_data = filled_df
BTC_data.head(10)

,Date,Open,High,Low,Close,Change,Amplitude,MA(7),MA(25),MA(99),Vol(BTC),Vol(USDT)
0,2019-12-01,7540.63,7541.85,7210.00,7390.89,-0.0200,0.0440,7409.45,8067.15,8881.46,60769.0,446525000.0
1,2019-12-02,7391.50,7420.56,7151.10,7294.28,-0.0131,0.0365,7435.84,7990.27,8852.68,46330.0,337889000.0
2,2019-12-03,7294.42,7400.00,7241.35,7292.71,-0.0002,0.0217,7455.35,7931.03,8821.58,33149.0,242379000.0
3,2019-12-04,7292.71,7750.00,7067.00,7194.32,-0.0135,0.0937,7410.47,7866.43,8791.37,83154.0,608671000.0
4,2019-12-05,7194.59,7485.00,7150.00,7389.00,0.0271,0.0466,7406.11,7800.41,8767.81,59307.0,434739000.0
5,2019-12-06,7389.00,7590.03,7305.00,7527.47,0.0187,0.0386,7375.79,7752.18,8747.90,48189.0,356906000.0
6,2019-12-07,7527.80,7619.62,7470.16,7488.21,-0.0052,0.0199,7368.13,7698.83,8726.73,31499.0,237001000.0
7,2019-12-08,7487.31,7564.00,7374.86,7510.11,0.0029,0.0253,7385.16,7648.15,8705.75,29857.0,223314000.0
8,2019-12-09,7510.11,7650.00,7273.00,7338.64,-0.0228,0.0502,7391.49,7595.83,8681.64,46622.0,347471000.0
9,2019-12-10,7338.64,7407.60,7157.10,7224.13,-0.0156,0.0341,7381.70,7545.92,8650.17,49724.0,363127000.0


In [8]:
BTC_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1686 entries, 0 to 1685
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       1686 non-null   datetime64[ns]
 1   Open       1686 non-null   float64       
 2   High       1686 non-null   float64       
 3   Low        1686 non-null   float64       
 4   Close      1686 non-null   float64       
 5   Change     1686 non-null   float64       
 6   Amplitude  1686 non-null   float64       
 7   MA(7)      1686 non-null   float64       
 8   MA(25)     1686 non-null   float64       
 9   MA(99)     1686 non-null   float64       
 10  Vol(BTC)   1686 non-null   float64       
 11  Vol(USDT)  1686 non-null   float64       
dtypes: datetime64[ns](1), float64(11)
memory usage: 158.2 KB


In [9]:
BTC_data.to_csv(r'C:\Users\Administrator\Desktop\DSP391m_cryptocurrency-price-prediction\data\processed\BTC\BTC_data.csv', index=False)